# __Brough Lecture Notes: GARCH Models - Estimation via MLE__

<br>

Finance 5330: Financial Econometrics <br>
Tyler J. Brough <br>
Last Updated: March 28, 2019 <br>
<br>
<br>

These notes are based in part on the excellent monograph [Introduction to Python for Econometrics, Statistics, and Data Analysis](https://www.kevinsheppard.com/images/b/b3/Python_introduction-2016.pdf) by the econometrician Kevin Sheppard of Oxford Univeristy. Many thanks to Dr. Sheppard for making his lecture material publically available. 

<br>

## Estimating GARCH Models

The standard procedure to fit GARCH models to historical returns time series data is to implement a numerical _maximum likelihood estimation_ (MLE) method. 

<br>

__NB:__ though [Bayesian](https://www.springer.com/us/book/9783540786566) methods have been shown to be superior!

<br>

The typical setup is as follows: 

* Continuously compounded returns are assumed to have a conditionally normal distribution $N(0, \sigma_{t})$

* We can estimate the GARCH parameter weights via a numerical optimization routine such as Nelder-Mead or Newton-Raphson.

* That is, the numerical routine searches for the parameter values that maximizes the value of the likelihood function.

<br>

Under the normality assumption the probility density of $\epsilon_{t}$, conditional on $\sigma_{t}$, is 

<br>

$$
f(\epsilon | \sigma_{t}) = \frac{1}{\sqrt{2\pi \sigma_{t}}} e^{-0.5  \frac{\epsilon_{t}^{2}}{\sigma_{t}}}
$$

<br>

Since the $\epsilon_{t}$ are conditionally independent, the probability of observing the actual returns that are observed is  the product of the probabilities, this is given by the likelihood function:

<br>

$$
\prod\limits_{t=1}^{T} f(\epsilon_{t} | \sigma_{t}) = \prod\limits_{t=1}^{T} \left( \frac{1}{\sqrt{2\pi \sigma_{t}}} e^{-0.5  \frac{\epsilon_{t}^{2}}{\sigma_{t}}} \right)
$$

<br>

For the GARCH(1,1) model, $\sigma_{t}$ is a function of $\omega$, $\alpha$, and $\beta$. The MLE will select values for these parameters $\hat{\omega}$, $\hat{\alpha}$, and $\hat{\beta}$ - that maximize the value of the probability of observing the returns we actually historically did observe. 

<br>

Typically, it is easiest to maximize the value of the log-likelihood function as follows: 

<br>

$$
\sum\limits_{t=1}^{T} \left[ -0.5 \ln{(\sigma_{t})} - 0.5 \frac{\epsilon_{t}^{2}}{\sigma_{t}}\right]
$$

We can omit the term $-0.5 \ln{(2 \pi)}$ since it does not affect the solution. Though sometimes it is left in. 

<br>

We can implement this MLE estimation in Python by utilizing the [optimize](https://docs.scipy.org/doc/scipy/reference/optimize.html) module in [Scipy](https://docs.scipy.org/doc/scipy/reference/index.html), which contains a host of numerical optimization routines. 

<br>

First, we need to define a function to implement the log-likelihood function.

<br>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]

import seaborn
from numpy import size, log, exp, pi, sum, diff, array, zeros, diag, mat, asarray, sqrt, copy
from numpy.linalg import inv

from scipy.optimize import fmin_slsqp

In [4]:
def garch_likelihood(params, data, sigma2, out=None):
    mu = params[0]
    omega = params[1]
    alpha = params[2]
    beta = params[3]
    
    T = size(data, 0)
    eps = data - mu
    
    for t in range(1, T):
        sigma2[t] = omega + alpha * eps[t-1]**2 + beta * sigma2[t-1]
        
    lls = 0.5 * (log(2 * pi) + log(sigma2) + eps**2/sigma2)
    ll = sum(logliks)
    
    if out is None:
        results = ll
    else:
        results = (ll, lls, copy(sigma2))
        
    return results